# Homework and bake-off: Relation extraction using distant supervision

In [ ]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [2]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [3]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [4]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [5]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [6]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [7]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

In [51]:
splits['tiny'].corpus.examples[0]

Example(entity_1='Island', entity_2='Indonesia', left='Date : Jul 2002 Location : Vienna Posts : 18,777 Re : 10,000 $ week of May5th , Entries 10,000 $ Tarakan , Indonesia Walikota Tarakan Open 2007 Surface : Hard Tarakan is an', mention_1='island', middle='in the East Kalimantan ,', mention_2='Indonesia', right='. It is a marshy island situated in the eastern Celebes Sea , off the northeastern coast of Borneo . The island occupies an area', left_POS='Date/NN :/: Jul/NNP 2002/CD Location/NNP :/: Vienna/NNP Posts/NNPS :/: 18,777/CD Re/NNP :/: 10,000/CD $/$ week/NN of/IN May5th/NN ,/, Entries/NNS 10,000/CD $/$ Tarakan/JJ ,/, Indonesia/NNP Walikota/NNP Tarakan/NNP Open/NNP 2007/CD Surface/NN :/: Hard/JJ Tarakan/NNP is/VBZ an/DT', mention_1_POS='island/NN', middle_POS='in/IN the/DT East/NNP Kalimantan/NNP ,/,', mention_2_POS='Indonesia/NNP', right_POS='./. It/PRP is/VBZ a/DT marshy/JJ island/NN situated/VBN in/IN the/DT eastern/JJ Celebes/NNP Sea/NNP ,/, off/IN the/DT northeastern/JJ coast

## Baselines

### Hand-build feature functions

In [8]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [9]:
featurizers = [simple_bag_of_words_featurizer]

In [10]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [11]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.889      0.376      0.699        340       5716
author                    0.782      0.564      0.726        509       5885
capital                   0.613      0.200      0.434         95       5471
contains                  0.790      0.602      0.743       3904       9280
film_performance          0.771      0.559      0.717        766       6142
founders                  0.761      0.403      0.646        380       5756
genre                     0.609      0.165      0.395        170       5546
has_sibling               0.854      0.246      0.572        499       5875
has_spouse                0.854      0.325      0.644        594       5970
is_a                      0.667      0.213      0.468        497       5873
nationality               0.566      0.156      0.371        301       5677
parents     

Studying model weights might yield insights:

In [12]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.557 Córdoba
     2.490 Taluks
     2.417 Valais
     ..... .....
    -1.105 for
    -1.240 Europe
    -1.478 America

Highest and lowest feature weights for relation author:

     3.285 author
     2.459 entitled
     2.457 book
     ..... .....
    -2.202 or
    -2.998 1852
    -4.079 1945

Highest and lowest feature weights for relation capital:

     3.265 capital
     1.927 especially
     1.789 km
     ..... .....
    -1.207 Roman
    -1.566 ’
    -2.209 Madras

Highest and lowest feature weights for relation contains:

     2.643 bordered
     2.553 third-largest
     2.018 tiny
     ..... .....
    -2.572 who
    -2.828 Mile
    -3.779 Ceylon

Highest and lowest feature weights for relation film_performance:

     3.902 starring
     3.602 alongside
     3.478 opposite
     ..... .....
    -1.904 Tamil
    -1.987 Iruvar
    -3.860 Mohabbatein

Highest and lowest feature weights for relation founders:

     4.090 fo

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [13]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [14]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [15]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [16]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.877      0.441      0.732        340       5716
author                    0.847      0.446      0.718        509       5885
capital                   0.564      0.232      0.438         95       5471
contains                  0.645      0.417      0.581       3904       9280
film_performance          0.805      0.322      0.619        766       6142
founders                  0.775      0.245      0.541        380       5756
genre                     0.611      0.065      0.227        170       5546
has_sibling               0.854      0.259      0.585        499       5875
has_spouse                0.880      0.357      0.680        594       5970
is_a                      0.766      0.145      0.412        497       5873
nationality               0.600      0.219      0.445        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [23]:
def run_svm_model_factory():
    from sklearn.svm import SVC
    ##### YOUR CODE HERE
    model_factory = lambda: SVC(kernel='linear')
    
    svc_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_factory,
        verbose=True)
    
    return svc_results




In [24]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [25]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.836      0.344      0.650        340       5716
author                    0.750      0.619      0.720        509       5885
capital                   0.667      0.274      0.518         95       5471
contains                  0.775      0.606      0.734       3904       9280
film_performance          0.738      0.629      0.713        766       6142
founders                  0.728      0.437      0.642        380       5756
genre                     0.573      0.253      0.457        170       5546
has_sibling               0.783      0.238      0.537        499       5875
has_spouse                0.850      0.352      0.662        594       5970
is_a                      0.585      0.264      0.470        497       5873
nationality               0.533      0.189      0.391        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [27]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
svc_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers = [directional_bag_of_words_featurizer],
        model_factory=model_factory,
        verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.902      0.406      0.725        340       5716
author                    0.837      0.595      0.774        509       5885
capital                   0.641      0.263      0.498         95       5471
contains                  0.808      0.674      0.777       3904       9280
film_performance          0.829      0.658      0.788        766       6142
founders                  0.797      0.413      0.672        380       5756
genre                     0.741      0.235      0.518        170       5546
has_sibling               0.870      0.255      0.586        499       5875
has_spouse                0.869      0.359      0.677        594       5970
is_a                      0.764      0.247      0.539        497       5873
nationality               0.618      0.209      0.444        301       5677
parents     

In [28]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [150]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #get a list of bigram str 
        #count appearance of each bigram pair 
        bigram_list = get_tag_bigrams(ex.middle_POS)
        for bigram in bigram_list:
            feature_counter[bigram] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        bigram_list = get_tag_bigrams(ex.middle_POS)   
        for bigram in bigram_list:
            feature_counter[bigram] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    #a list of tags from sentence 
    result =[]
    tag = get_tags(s)    
    for i in zip([start_symbol]+tag, tag+[end_symbol]):
        result.append(i[0] + ' ' +i[1])
        
    return result

    
def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
bigram_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers = [middle_bigram_pos_tag_featurizer],
        model_factory=model_factory,
        verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.341      0.650        340       5716
author                    0.708      0.334      0.579        509       5885
capital                   0.576      0.200      0.419         95       5471
contains                  0.747      0.595      0.711       3904       9280
film_performance          0.719      0.454      0.644        766       6142
founders                  0.543      0.166      0.373        380       5756
genre                     0.580      0.171      0.392        170       5546
has_sibling               0.680      0.170      0.425        499       5875
has_spouse                0.748      0.269      0.552        594       5970
is_a                      0.585      0.187      0.410        497       5873
nationality               0.418      0.076      0.221        301       5677
parents     

In [151]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [152]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [250]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj): 
        #get a list of synsets
        synset_list = get_synsets(ex.middle_POS)
        #flatten the list 
        flatten_synset = [item for items in synset_list for item in items]
        for synet in flatten_synset:
            feature_counter[str(synet)] += 1
            
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        synset_list = get_synsets(ex.middle_POS)   
        flatten_synset = [item for items in synset_list for item in items]
        for synet in flatten_synset:
            feature_counter[str(synet)] += 1

    return feature_counter



def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    converted_wt = []
    for i in wt:
        converted_tag = convert_tag(i[1])
        word =i[0]
        converted_wt.append([word,converted_tag])
        
    return [wn.synsets(i[0], pos=i[1]) for i in converted_wt]

    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'J':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
bigram_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers = [synset_featurizer],
        model_factory=model_factory,
        verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.797      0.335      0.625        340       5716
author                    0.789      0.456      0.688        509       5885
capital                   0.667      0.232      0.485         95       5471
contains                  0.773      0.587      0.727       3904       9280
film_performance          0.790      0.555      0.728        766       6142
founders                  0.728      0.387      0.619        380       5756
genre                     0.455      0.206      0.366        170       5546
has_sibling               0.809      0.220      0.527        499       5875
has_spouse                0.862      0.305      0.631        594       5970
is_a                      0.581      0.231      0.446        497       5873
nationality               0.494      0.136      0.324        301       5677
parents     

In [251]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [252]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# Enter your system description in this cell.
# Please do not remove this comment.

'''
From my own experience, desigining better feature will give the relation extraction system a boost than 
stacking comlicated Machine Learning Models.
So the focus of my system will be on experiencing different featurizers.

Featurizer ideas: 
1. So far we are just buidling feature based on the [mideel] and [middle_POS], we should be using the 
same techniques to see if we can identify some common pattern on the [left] or [right] words. 
2. From bidirectional give a simple system a noticible boost, so this technique will be applied to most 
of the featurizers. 
3. We have tried bigram, should also try trigram ? 
4. From experience, SCV don't give a good result and took a long time to train, so i will be mainly using 
logistic regression for modeling. If have time will try random forest. 


'''

In [259]:
#1. try on stacking existing featurizer 
featurizers_1 = [simple_bag_of_words_featurizer,directional_bag_of_words_featurizer,
                 middle_bigram_pos_tag_featurizer]
model_factory_1 = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers_1,
    model_factory=model_factory_1,
    #vectorize=False,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.874      0.388      0.699        340       5716
author                    0.850      0.644      0.799        509       5885
capital                   0.700      0.295      0.549         95       5471
contains                  0.837      0.674      0.799       3904       9280
film_performance          0.827      0.701      0.799        766       6142
founders                  0.785      0.432      0.674        380       5756
genre                     0.660      0.365      0.568        170       5546
has_sibling               0.875      0.267      0.601        499       5875
has_spouse                0.845      0.375      0.676        594       5970
is_a                      0.746      0.342      0.603        497       5873
nationality               0.581      0.262      0.467        301       5677
parents     

In [258]:
#2. try different model
from sklearn.ensemble import RandomForestClassifier
model_factory_2 = lambda: RandomForestClassifier(max_depth=2, random_state=0) #

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers_1,
    model_factory=model_factory_2,
    #vectorize=False,
    verbose=True)
#:( bad result, need more tuning 

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   1.000      0.006      0.029        340       5716
author                    0.000      0.000      0.000        509       5885
capital                   0.000      0.000      0.000         95       5471
contains                  0.940      0.036      0.156       3904       9280
film_performance          0.000      0.000      0.000        766       6142
founders                  0.000      0.000      0.000        380       5756
genre                     0.000      0.000      0.000        170       5546
has_sibling               0.000      0.000      0.000        499       5875
has_spouse                0.000      0.000      0.000        594       5970
is_a                      0.000      0.000      0.000        497       5873
nationality               0.000      0.000      0.000        301       5677
parents     

/Users/catherinewang/opt/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

#1. try on stacking existing featurizer 
featurizers_1 = [simple_bag_of_words_featurizer,directional_bag_of_words_featurizer,
                 middle_bigram_pos_tag_featurizer]
model_factory_1 = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers_1,
    model_factory=model_factory_1,
    vectorize=False,
    verbose=True)


## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE




In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE


